In [1]:
!pip install azure-ai-ml azure-identity

  Using cached azure_ai_ml-1.27.1-py3-none-any.whl.metadata (37 kB)
  Using cached azure_identity-1.23.0-py3-none-any.whl.metadata (81 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached jsonschema-4.24.0-py3-none-any.whl.metadata (7.8 kB)
  Using cached strictyaml-1.7.3-py3-none-any.whl.metadata (11 kB)
  Using cached azure_storage_blob-12.25.1-py3-none-any.whl.metadata (26 kB)
  Using cached azure_storage_file_share-12.21.0-py3-none-any.whl.metadata (50 kB)
  Using cached azure_storage_file_datalake-12.20.0-py3-none-any.whl.metadata (16 kB)
  Using cached pydash-8.0.5-py3-none-any.whl.metadata (4.5 kB)
  Using cached azure_monitor_opentelemetry-1.6.10-py3-none-any.whl.metadata (21 kB)
  Using cached jsonschema_specifications-2025.4.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.25.1-cp311-cp311-win_amd64.whl.metadata (4.2 kB)
  Using cached azure_core_tracing_opente

In [1]:
from azure.ai.ml import MLClient
from azure.identity import InteractiveBrowserCredential

# Create a browser credential
credential = InteractiveBrowserCredential()

# subscription_id, resource_group, and workspace_name are strings
subscription_id = "0a94de80-6d3b-49f2-b3e9-ec5818862801"
resource_group = "buas-y2"
workspace_name = "Y2D-Staff-2025"

# Create an MLClient using the credential and workspace details
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

In [2]:
# List the compute s
compute_targets = ml_client.compute.list()
for compute_target in compute_targets:
    print(compute_target.name, ":", compute_target.type)

# List the environments
environments = ml_client.environments.list()
for environment in environments:
    print(environment.name, ":", environment.latest_version)


adsai-lambda-0 : kubernetes
Example : 2
training-environment-test : 1
training-environment-sdkv2 : 1.0.4
training-environment : 1
AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu : 10


In [3]:
from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential

data_asset = ml_client.data.get("transcripts_train", version="1")

print("Data Asset ID:", data_asset.id)
print("Data Asset Name:", data_asset.name)
print("Data Asset Version:", data_asset.version)
# URI
print("Data Asset URI:", data_asset.path)

Data Asset ID: /subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourceGroups/buas-y2/providers/Microsoft.MachineLearningServices/workspaces/Y2D-Staff-2025/data/transcripts_train/versions/1
Data Asset Name: transcripts_train
Data Asset Version: 1
Data Asset URI: azureml://subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/workspaces/Y2D-Staff-2025/datastores/workspaceblobstore/paths/LocalUpload/1e64f88687dc0ea5f9520fb642cf8b30/train.csv


In [4]:
from azure.ai.ml import command
from azure.ai.ml import Input

# Get the environment and compute target
# Replace with your environment and compute target from the previous step
env = ml_client.environments.get("training-environment-sdkv2", "1.0.4")  # 📝 replace with your environment name and version
compute_target = "adsai-lambda-0"  # 📝 replace with your compute target
# This is the path to the data folder in the datastore, click on the data asset in the Azure ML portal to get the path
path = "azureml://subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/workspaces/Y2D-Staff-2025/datastores/workspaceblobstore/paths/LocalUpload/1e64f88687dc0ea5f9520fb642cf8b30/train.csv"


In [11]:
# Setup the command job
job = command(
    inputs=dict(
        data=Input(type="uri_file", description="Data asset URI", path=path),
    ),
    code="../backend-microservice/src/backend_microservice/",  # location of source code
    command="python train.py --use-uri --data-path ${{inputs.data}}",
    environment=env,
    compute=compute_target,
    display_name="train LoRA model with data",
)

In [ ]:
ml_client.create_or_update(job)

Uploading backend_microservice (0.04 MBs): 100%|##########| 39801/39801 [00:00<00:00, 261156.49it/s]




Experiment,Name,Type,Status,Details Page
notebooks,boring_plane_2j8ltknns9,command,Starting,Link to Azure Machine Learning studio


: 

In [12]:
import os
os.getcwd()

'c:\\Users\\deanv\\Dropbox\\0_Buas\\2024-2025\\Y2D\\Demo Project\\mlops-demo-project\\notebooks'

In [15]:
import os
print("Current working directory:", os.getcwd())
print("Contents of parent directory:")
for item in os.listdir(".."):
    print(f"  {item}")

# Check if the path exists
code_path = "../backend-microservice/src/backend-microservice/"
print(f"Path exists: {os.path.exists(code_path)}")

Current working directory: c:\Users\deanv\Dropbox\0_Buas\2024-2025\Y2D\Demo Project\mlops-demo-project\notebooks
Contents of parent directory:
  .git
  .github
  .gitignore
  backend-microservice
  data
  frontend
  NEW_FEATURES_TO_IMPLEMENT.MD
  notebooks
  README.md
Path exists: False


In [16]:
import os

# Check contents of backend-microservice directory
backend_path = "../backend-microservice"
print(f"Contents of {backend_path}:")
for item in os.listdir(backend_path):
    item_path = os.path.join(backend_path, item)
    if os.path.isdir(item_path):
        print(f"  📁 {item}/")
    else:
        print(f"  📄 {item}")

# If there's a src directory, check its contents
src_path = "../backend-microservice/src"
if os.path.exists(src_path):
    print(f"\nContents of {src_path}:")
    for item in os.listdir(src_path):
        item_path = os.path.join(src_path, item)
        if os.path.isdir(item_path):
            print(f"  📁 {item}/")
        else:
            print(f"  📄 {item}")

Contents of ../backend-microservice:
  📄 .coverage
  📄 .pre-commit-config.yaml
  📁 .pytest_cache/
  📁 .venv/
  📄 docker-compose.yml
  📄 Dockerfile
  📁 docs/
  📁 htmlcov/
  📄 poetry.lock
  📄 pyproject.toml
  📄 README.md
  📁 src/
  📁 tests/

Contents of ../backend-microservice/src:
  📁 backend_microservice/


In [9]:

from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes

data_asset = ml_client.data.get("transcripts_train", version="1")

# to successfully create a job, customize the parameters below based on your workspace resources
job = command(
        command='ls "${{inputs.data}}"',
        inputs={
          "data": Input(path=data_asset.id,
              type=AssetTypes.URI_FILE,
              mode=InputOutputModes.RO_MOUNT
              )
          },
        environment="azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
        compute=compute_target,
        instance_type="defaultinstancetype",
        display_name="list data asset files",
      )

returned_job = ml_client.jobs.create_or_update(job)